# Graphical Models for Textual Data
This shows how graphical models can be used to infer relationships between textual data.

In [35]:
import pandas as pd
from regain.utils import flatten
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [1]:
filename = "../regain/data/text/webkb-train-stemmed.txt"
train = pd.read_csv(filename, header=None, sep='\t', index_col=0).dropna()
train.columns = ['words']
documents = train.words.fillna("")

In [2]:
words = np.unique(flatten([words.split(' ') for words in train.words.tolist() if isinstance(words, str)]))

ld = [dict(zip(*np.unique(row.words.split(' '), return_counts=True))) for row in train.itertuples()
      if isinstance(row.words, str)]

X = pd.DataFrame(ld, index=[row.Index for row in train.itertuples()
      if isinstance(row.words, str)]).fillna(0)

y = X.index

In [5]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)

df_tf = pd.DataFrame(tf.todense(), index=train.index, columns=tf_vectorizer.get_feature_names())

In [55]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    topics = []
    for topic_idx, topic in enumerate(H):
        topics.append(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
        print ("Topic %d: %s" % (topic_idx, topics[-1]))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for i, doc_index in enumerate(top_doc_indices):
            print ("doc %d: %s" % (doc_index, documents[doc_index]))
    return topics
# # NMF is able to use tf-idf
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
# tfidf = tfidf_vectorizer.fit_transform(documents)
# tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# # Run NMF
# nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
# nmf_W = nmf_model.transform(tfidf)
# nmf_H = nmf_model.components_

# print("NMF Topics")
# display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)

In [49]:
n_topics = 50
n_top_words = 3
n_top_documents = 3

In [47]:
# Run LDA
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5, learning_method='online',
                                      learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [56]:
print("LDA Topics")
topics = display_topics(lda_H, lda_W, tf_vectorizer.get_feature_names(), documents, n_top_words, n_top_documents)

df = pd.DataFrame(lda_W, index=train.index, columns=topics)
X = lda_W
y = documents.index

LDA Topics
Topic 0: databas washington queri
doc 971: seq project queri sequenc data document construct time put order databas order time put databas time put databas order document content project object current statu motiv seq data model sequin queri languag optim techniqu seq system develop public relat work contact inform project object number import databas applic requir process larg amount order sequenc data domain applic includ financi manag histor analysi econom social scienc medic scienc biolog scienc exist relat databas inadequ regard data collect treat set sequenc express sequenc queri tediou evalu ineffici databas model data abstract sequenc data sequenc queri declar manner util order semant advantag uniqu opportun queri optim evalu integr sequenc data relat data user store queri combin relat sequenc requir serv goal seq project kind sequenc support tempor sequenc import kind queri express notion previou natur sequenc queri optim evalu effici issu studi theori databas syste

In [11]:
# X = pd.DataFrame([], columns=words)
# y = []
# for row in train.itertuples():
#     if isinstance(row.words, str):
#         series = pd.Series(dict(zip(*np.unique(row.words.split(' '), return_counts=True))), name=row.Index)
#         X = X.append(series)
#         y.append(row.Index)

# X = X.fillna(0)
# X.index = y

# y = np.asarray(y)

words_to_discard = []
for yy in np.unique(y):
    words_to_discard += list(X[words[X[y==yy].sum(axis=0) == 0]].columns)

words_to_keep = list(set(X.columns) - set(words_to_discard))

def logentropy_normalize(X):
    P = X / X.values.sum(axis=0, keepdims=True)
    E = 1 + (P * np.log(P)).fillna(0).values.sum(axis=0, keepdims=True) / np.log(X.shape[0] + 1)
    return E * np.log(1 + X)

X_new = logentropy_normalize(X)[words_to_keep]

/Users/federicot/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log


In [12]:
from gensim.models import LogEntropyModel
from gensim.test.utils import common_texts
from gensim.corpora import Dictionary

corp = [w.split(' ') for w in train.words.tolist() if isinstance(w, str)]
text = corp #common_texts
dct = Dictionary(text)  # fit dictionary

num_terms = 50 # or words.size
dct.filter_extremes(keep_n=num_terms)

corpus = [dct.doc2bow(row) for row in text][:10]  #convert to BoW format
model = LogEntropyModel(corpus, normalize=True)  # fit model

In [14]:
import ctmmodel
ctm_model = ctmmodel.CtmModel(corpus, id2word=dct, num_topics=15)

0


/Users/federicot/src/fdtomasi/regain/notebooks/ctmmodel.py:235: RuntimeWarning: divide by zero encountered in log
  bound += np.sum(np.log(nu2)) + self.num_topics  # TODO safe_log
/Users/federicot/src/fdtomasi/regain/notebooks/ctmmodel.py:352: RuntimeWarning: divide by zero encountered in true_divide
  res += 0.5 / nu2


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [15]:
all_words = []
for c in corpus:
    doc_words = []
    for cc in c:
        doc_words.extend([dct[cc[0]]] * cc[1])
    all_words.append(' '.join(doc_words))

In [26]:
import pyctm
from pyctm import variational_bayes, inferencer, utils
# parameter set 3
alpha_mu=0.
alpha_sigma=1
alpha_beta=0

ctm_inferencer = variational_bayes.VariationalBayes();
ctm_inferencer._initialize(all_words, list(dct.values()), number_of_topics=15,
                           alpha_mu=alpha_mu, alpha_sigma=alpha_sigma, alpha_beta=alpha_beta);

for iteration in range(50):
    ctm_inferencer.learning(-1)

successfully parse 10 documents...
creating 8 processes
[[-64.209422]] nan
e_step and m_step of iteration 1 finished in 0.122496 and 0.000340939 seconds respectively with log likelihood nan
update hyper-parameter mu to [ 0.00236528 -0.02848685  0.05582271 -0.01741234  0.0204918  -0.0208173
 -0.04207808  0.04002519 -0.02070904  0.02731378  0.02718902  0.05141206
 -0.08558489  0.02507594 -0.04204119]
update hyper-parameter sigma to
[[ 4.72478977e-01 -3.39244515e-03 -2.62055756e-03 -1.44179434e-03
  -1.95299415e-03 -4.47710247e-04 -3.65385988e-03  2.11118227e-03
  -1.66125279e-03 -4.17026441e-03  1.02400313e-02  5.18919079e-03
  -6.07055816e-03 -6.90568613e-06 -1.57402068e-04]
 [-3.39244515e-03  4.81943480e-01 -1.58062357e-03 -8.30949061e-03
  -2.18674288e-03  4.04759563e-03  3.04638493e-03  1.66296854e-03
   5.57377647e-04 -7.01017261e-04 -6.64291342e-04 -5.91226488e-03
   1.45914450e-03  1.34757771e-05  1.37211109e-03]
 [-2.62055756e-03 -1.58062357e-03  4.59117474e-01  2.89237628e-03
  

/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)


[[nan]] nan
e_step and m_step of iteration 4 finished in 0.198572 and 0.000667095 seconds respectively with log likelihood nan
update hyper-parameter mu to [-0.16727981  0.32594371  0.05886509  0.49638625  0.01029289 -0.3300881
 -0.22656625 -0.08355682 -0.32836028 -0.18850041  0.72680491  0.14737153
 -0.42744742 -0.02372016 -0.30797697]
update hyper-parameter sigma to
[[ 3.04374596e-01 -2.63160302e-01 -2.27319180e-02  2.56799936e-04
  -1.21067384e-02 -3.86785123e-02 -4.13834620e-02 -8.17099385e-03
   2.61265530e-03 -3.36465052e-02  2.30791651e-01  7.07854877e-02
  -5.81173247e-03 -2.19276593e-02 -5.25972809e-03]
 [-2.63160302e-01  2.06402777e+00  9.39058622e-02 -4.74048113e-01
  -1.26360487e-01  1.69569367e-01  1.08186394e-01  8.12151773e-02
  -9.45061680e-04  3.70383844e-02 -8.81337417e-01 -2.85740607e-01
  -5.34624233e-02  2.17862866e-01  3.82619455e-02]
 [-2.27319180e-02  9.39058622e-02  2.20353026e-01  5.53731706e-02
   3.28856927e-02  2.14289952e-02  3.01577808e-02  1.48697184e-02

/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_b

[[nan]] nan
e_step and m_step of iteration 5 finished in 0.227302 and 0.000361919 seconds respectively with log likelihood nan
update hyper-parameter mu to [-0.1639434   0.57772119 -0.00592061  0.73896908  0.04172367 -0.3959841
 -0.223769   -0.11346379 -0.38556161 -0.20787965  0.59208357  0.09163749
 -0.4900268  -0.00500942 -0.34732992]
update hyper-parameter sigma to
[[ 3.90540065e-01 -4.79736260e-01 -3.93749278e-02  3.40554811e-02
  -1.87440005e-02 -6.95112156e-02 -7.69832871e-02 -1.94325138e-02
   1.89947406e-02 -6.18366722e-02  3.58904182e-01  8.63382884e-02
  -9.91995748e-03 -2.81262970e-02 -2.34910142e-02]
 [-4.79736260e-01  2.76395398e+00  1.49496144e-01 -4.43864100e-01
  -4.68919545e-02  2.56571887e-01  1.88046961e-01  1.19217477e-01
  -5.99303768e-02  1.23704956e-01 -1.39682298e+00 -3.03353305e-01
  -1.05878231e-01  2.48432962e-01  1.02343894e-01]
 [-3.93749278e-02  1.49496144e-01  1.96305938e-01  3.31636072e-02
   2.80638695e-02  2.70504670e-02  3.56832772e-02  1.73689210e-02

...

[[nan]] nan
e_step and m_step of iteration 48 finished in 0.253432 and 0.000555038 seconds respectively with log likelihood nan
update hyper-parameter mu to [-1.90997799  0.60093391  0.09876691  0.91009454 -0.54286361 -1.21453525
  0.14422736  0.11333985 -0.25020644  0.00916267 -0.13631577 -0.38821529
 -0.97295526 -0.41720154 -1.15308992]
update hyper-parameter sigma to
[[ 2.88001665e+00 -1.40151232e+00 -1.68273656e-02  4.52027913e-01
  -1.17118580e+00 -5.41255673e-01 -8.21568557e-01 -9.84474571e-02
   1.01059327e+00 -2.09237059e-01  1.70478610e+00 -3.55714374e-01
  -1.69398387e-01  4.51551283e-01 -1.29542374e+00]
 [-1.40151232e+00  2.83269016e+00  5.95605696e-02 -2.74304113e-01
   8.99211413e-01  5.10761554e-01  2.03717906e-01 -3.52725428e-02
  -8.19184831e-01  1.82543318e-01 -1.80807808e+00  1.25754230e-01
  -2.68233663e-01  9.80866622e-04  8.19202981e-01]
 [-1.68273656e-02  5.95605696e-02  8.15356688e-02 -4.13141638e-02
  -6.78719035e-03  8.78242309e-03  3.25418987e-03 -2.38353583e-

/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_b

[[nan]] nan
e_step and m_step of iteration 49 finished in 0.258397 and 0.000478268 seconds respectively with log likelihood nan
update hyper-parameter mu to [-1.96241824  0.59284573  0.09728396  0.9153697  -0.55246023 -1.22586813
  0.14850322  0.11195552 -0.25091048  0.01258692 -0.13933573 -0.39859721
 -0.97976468 -0.42452758 -1.17429317]
update hyper-parameter sigma to
[[ 2.95888670e+00 -1.42118912e+00 -1.48700647e-02  4.44943065e-01
  -1.18777952e+00 -5.50188255e-01 -8.07480435e-01 -9.63393105e-02
   1.01945636e+00 -2.10933954e-01  1.72825485e+00 -3.57620398e-01
  -1.58859512e-01  4.57375614e-01 -1.30132553e+00]
 [-1.42118912e+00  2.83414776e+00  5.85254295e-02 -2.77790563e-01
   8.96569672e-01  5.12954446e-01  1.91208347e-01 -3.62075898e-02
  -8.18616274e-01  1.82823763e-01 -1.80319200e+00  1.30332682e-01
  -2.69988217e-01  3.41066058e-03  8.20828740e-01]
 [-1.48700647e-02  5.85254295e-02  8.04638053e-02 -4.02595717e-02
  -8.56144186e-03  8.08664732e-03  1.56330588e-03 -2.61100694e-

/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_bayes.py:229: RuntimeWarning: invalid value encountered in multiply
  numpy.exp(log_phi) * log_phi * term_counts)
/Users/federicot/src/PyCTM/pyctm/variational_b

[[nan]] nan
e_step and m_step of iteration 50 finished in 0.260164 and 0.000482082 seconds respectively with log likelihood nan
update hyper-parameter mu to [-2.01139391  0.58317858  0.09545674  0.92444535 -0.56215906 -1.23766784
  0.15254806  0.11024599 -0.25016808  0.01544997 -0.14190671 -0.41099179
 -0.98753846 -0.43144622 -1.20029562]
update hyper-parameter sigma to
[[ 3.02427903e+00 -1.44089274e+00 -1.25332282e-02  4.29736777e-01
  -1.19716958e+00 -5.56304397e-01 -7.81484180e-01 -9.24179301e-02
   1.02094675e+00 -2.11764779e-01  1.74317900e+00 -3.58396019e-01
  -1.41914610e-01  4.57824757e-01 -1.28963832e+00]
 [-1.44089274e+00  2.83791263e+00  5.74098521e-02 -2.81475183e-01
   8.92435759e-01  5.15052620e-01  1.75959356e-01 -3.74761749e-02
  -8.17679868e-01  1.83416844e-01 -1.79649412e+00  1.37324288e-01
  -2.72481640e-01  6.73593675e-03  8.20865519e-01]
 [-1.25332282e-02  5.74098521e-02  7.93658932e-02 -3.89541244e-02
  -1.04633455e-02  7.31385098e-03 -3.47982274e-04 -2.87139270e-

In [27]:
logl, lamda, nu = ctm_inferencer.inference(all_words)

ll = utils.topic_beta(ctm_inferencer)

topic_words = pd.DataFrame(ll)

successfully parse 10 documents...


In [32]:
ll

[{'languag': 0.9550024041822259,
  'email': 0.04499759581777409,
  'hour': 0.0,
  'area': 0.0,
  'teach': 0.0,
  'student': 0.0,
  'design': 0.0,
  'program': 0.0,
  'list': 0.0,
  'time': 0.0,
  'project': 0.0,
  'distribut': 0.0,
  'fall': 0.0,
  'current': 0.0,
  'base': 0.0,
  'softwar': 0.0,
  'mail': 0.0,
  'algorithm': 0.0,
  'link': 0.0,
  'develop': 0.0,
  'gener': 0.0,
  'group': 0.0,
  'web': 0.0,
  'updat': 0.0,
  'includ': 0.0,
  'model': 0.0,
  'inform': 0.0,
  'interest': 0.0,
  'data': 0.0,
  'professor': 0.0,
  'offic': 0.0,
  'applic': 0.0,
  'schedul': 0.0,
  'parallel': 0.0,
  'home': 0.0,
  'graduat': 0.0,
  'engin': 0.0,
  'fax': 0.0,
  'process': 0.0,
  'technolog': 0.0,
  'network': 0.0,
  'assist': 0.0,
  'class': 0.0,
  'work': 0.0,
  'problem': 0.0,
  'system': 0.0,
  'public': 0.0,
  'phone': 0.0,
  'paper': 0.0,
  'address': 0.0},
 {'offic': 0.17683265141400326,
  'mail': 0.16961588788988885,
  'program': 0.16961588788988885,
  'gener': 0.1388697436260792,


In [28]:
topic_str_repr = []
for row in topic_words.iterrows():
    print(pd.DataFrame(row[1].sort_values(ascending=False)[:3]).T)
    topic_str_repr.append(' '.join(row[1].sort_values(ascending=False)[:3].index))

    languag     email  inform
0  0.955002  0.044998     0.0
      offic      mail   program
1  0.176833  0.169616  0.169616
   professor   project  distribut
2   0.641834  0.358166        0.0
       link      home      list
3  0.229816  0.147937  0.107528
   student  problem  work
4      0.5      0.5   0.0
   engin  work  email
5    1.0   0.0    0.0
      paper     email  work
6  0.528642  0.471358   0.0
   phone  work  email
7    1.0   0.0    0.0
   inform    work  email
8  0.9304  0.0696    0.0
   offic  work  email
9    1.0   0.0    0.0
    parallel     model    public
10   0.20256  0.175595  0.118599
         web  distribut  email
11  0.809998   0.190002    0.0
       group   graduat   softwar
12  0.475367  0.174878  0.174878
      assist     updat  work
13  0.697059  0.302941   0.0
         fax   address  email
14  0.988735  0.011265    0.0


In [29]:
word_dct_values = list(dct.values())

dff = pd.DataFrame(ctm_model.beta, columns=word_dct_values)
dff = dff[sorted(dff.columns)]

In [34]:
dff

,address,algorithm,applic,area,assist,base,class,current,data,design,...,schedul,softwar,student,system,teach,technolog,time,updat,web,work
0,4.297508e-28,4.442288e-13,1.594608e-58,0.0,3.626889e-28,3.907700e-35,9.228855e-59,4.035642e-59,1.331595e-43,2.307814e-34,...,3.660534e-34,6.221078e-61,1.361251e-23,9.341098e-13,2.620840e-58,2.350242e-43,0.0,3.400604e-43,7.610733e-16,4.908232e-58
1,1.086065e-27,3.042558e-13,1.112333e-57,0.0,6.809378e-28,3.429169e-34,1.286617e-57,8.110761e-59,6.945124e-44,9.500631e-34,...,1.088641e-34,1.272582e-57,1.286415e-22,2.524302e-12,5.519617e-58,2.224009e-43,0.0,6.674937e-43,1.978509e-17,5.927602e-58
2,1.884114e-28,7.210961e-14,6.429234e-58,0.0,2.121613e-28,4.698312e-34,3.240783e-58,1.763183e-58,4.682648e-43,4.425781e-34,...,3.573163e-34,6.573493e-58,4.487696e-23,8.529095e-13,2.662254e-58,4.034828e-43,0.0,6.844577e-43,3.584997e-16,4.153564e-58
3,1.201195e-27,1.189983e-12,7.382704e-58,0.0,6.240019e-28,5.481158e-34,6.340583e-59,2.364767e-58,3.685738e-43,5.534289e-35,...,1.114838e-34,3.194085e-58,2.069084e-23,8.630395e-13,1.684885e-59,6.503306e-43,0.0,2.929934e-43,4.061334e-16,9.278982e-58
4,1.840626e-27,2.941640e-12,1.475254e-57,0.0,2.624923e-27,1.755981e-33,1.487372e-57,8.136686e-58,2.340282e-42,9.507489e-34,...,1.353739e-33,1.882511e-57,4.065642e-23,3.575254e-12,3.412444e-58,1.130207e-42,0.0,2.640923e-42,1.682826e-15,6.666696e-58
5,2.655721e-27,2.671455e-12,4.089628e-57,0.0,5.166151e-27,2.267259e-33,3.247349e-57,1.101824e-57,6.065486e-44,1.131217e-33,...,2.906701e-33,2.795796e-57,2.932483e-22,5.224053e-12,2.955734e-57,2.980650e-42,0.0,4.066538e-42,5.811925e-15,2.940288e-57
6,6.479323e-28,3.999573e-13,4.822469e-58,0.0,2.121522e-28,3.335881e-34,3.283193e-58,4.474074e-58,2.233428e-43,3.729290e-35,...,3.571652e-35,3.631423e-58,2.200142e-23,3.827787e-13,2.802645e-58,5.016529e-43,0.0,7.368146e-45,8.034731e-16,2.651235e-58
7,5.516442e-28,3.455159e-13,1.136104e-58,0.0,6.021776e-28,7.121135e-35,2.856959e-59,3.877539e-58,5.281742e-43,8.267019e-35,...,2.010455e-34,5.300393e-58,4.665412e-23,7.206113e-13,2.080050e-58,2.543600e-43,0.0,9.684001e-44,6.724321e-16,2.996886e-58
8,3.356666e-28,3.811897e-13,3.794479e-58,0.0,1.548470e-28,2.101832e-34,2.151644e-58,4.446637e-58,1.493330e-43,2.508354e-34,...,2.557608e-34,4.501890e-58,1.596453e-23,6.388451e-13,2.055806e-58,6.343626e-43,0.0,7.492648e-44,8.258365e-16,1.575674e-58
9,3.263619e-28,9.784919e-14,6.648659e-58,0.0,3.233717e-28,1.247186e-35,2.209099e-58,2.242135e-58,8.335059e-43,2.753690e-34,...,2.748975e-34,7.045917e-58,1.735251e-23,1.455828e-13,5.810166e-58,4.175769e-44,0.0,3.718986e-44,8.432158e-16,6.048442e-60


In [30]:
# from gensim import models
# model = models.LdaModel(corpus, id2word=dct, num_topics=num_terms)

In [31]:
import gensim
XX = gensim.matutils.corpus2dense(model[corpus], num_terms=num_terms).T
cols = list(dct.values())
df = pd.DataFrame(XX, columns=cols, index=y)
# df[words_to_keep].T.sort_index().T

# X = df[words_to_keep].values
X = df.values
y = df.index

ValueError: Shape of passed values is (10, 50), indices imply (2785, 50)

In [ ]:
from regain.covariance import kernel_time_graphical_lasso_
from regain.model_selection import stability_optimization
from sklearn.model_selection import StratifiedShuffleSplit

mdl = kernel_time_graphical_lasso_.KernelTimeGraphicalLasso(
    verbose=0, kernel=np.ones((np.unique(y).size, np.unique(y).size)), psi='l1',
    alpha=0.45, max_iter=1000).fit(X, y)

In [ ]:
socv = stability_optimization.GraphicalModelStabilitySelection(
    mdl, param_grid=dict(alpha=np.logspace(2, -2)), cv=StratifiedShuffleSplit(100)
).fit(X, y)

In [ ]:
n_times = np.unique(y).size
n_dim = X.shape[1]

In [ ]:
idx = np.triu_indices(n_dim, 1)
dof = idx[0].size * n_times

In [ ]:
from regain import utils
# utils.save_pickle(socv, "socv")

socv = utils.load_pickle("socv.pkl")

In [ ]:
mdl = socv.best_estimator_

In [ ]:
print("Nonzero percentage: %.4f" % (np.sum([np.count_nonzero(P[idx]) for P in mdl.precision_]) / dof))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from kdge import plot_plotly
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools

py.init_notebook_mode()
import plotly.io as pio

In [ ]:
p = mdl.precision_[0]

In [ ]:
from regain.utils import retain_top_n

In [ ]:
k = -1
trace = []
graphs = []
# for i, p in enumerate(ltgl.precision_ - ltgl.latent_):
for i, p in enumerate(mdl.precision_):

    A = np.abs(p - np.diag(np.diag(p)))
    A = retain_top_n(A, top_n)
    G = nx.from_numpy_matrix(A * 0.00001)
    graphs.append(G)
    trace.append(pl.plot_circular(G, df.columns, 1.4, cmap='Blues',
                                  #color_nodes=plt.rcParams['axes.prop_cycle'].by_key()['color'][:5]
                                 ))

fig = tools.make_subplots(
    rows=1, cols=len(mdl.precision_), horizontal_spacing=.1, print_grid=False)

for j, tr in enumerate(trace):
    tr['data'][k]['xaxis'] = 'x' + str(j+1)
    tr['data'][k]['yaxis'] = 'y' + str(j+1)

for j, tr in enumerate(trace):
    for i, x in enumerate(tr['data']):
        col = j + 1
        x['legendgroup'] = 'group'+ str(j+1)
        x['showlegend'] = False
        fig.append_trace(x,1,col)
        


for j, tr in enumerate(trace):
    fig.layout.annotations += tuple([pl._set_ref(
        x, 'x'+ str(j+1), 'y'+ str(j+1)) for x in tr['layout']['annotations']])

    fig['layout']['xaxis'+str(j+1)].update(showgrid=False, zeroline=False, showticklabels=False)
    fig['layout']['yaxis'+str(j+1)].update(showgrid=False, zeroline=False, showticklabels=False)

py.init_notebook_mode()

fig['layout'].update(height=900, width=4000,hovermode='closest',
                     paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')
# fig.layout.annotations += tuple([dict(
#     text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
#     showarrow=False, xref="paper", yref="paper", x=0.005, y=-0.2)])
# fig['layout'].update(scene=dict(aspectmode="data"))
py.iplot(fig)
# py.offline.iplot(fig, filename='figure_factory_subplot', image='svg')

In [ ]:
pio.write_image(fig, "graphs.pdf")

In [ ]:
import pyctm

from pyctm import variational_bayes, inferencer

# parameter set 3
alpha_mu=0.
alpha_sigma=.1
alpha_beta=-.1

ctm_inferencer = variational_bayes.VariationalBayes();
ctm_inferencer._initialize(train.words.tolist(), words, 20, alpha_mu, alpha_sigma, alpha_beta);

for iteration in range(50):
    ctm_inferencer.learning(-1)

logl, lamda, nu = ctm_inferencer.inference(train.words.tolist())

ll = utils.topic_beta(ctm_inferencer)

topic_words = pd.DataFrame(ll)

In [ ]:
np.abs(lamda).sum()

In [ ]:
from pyctm import utils

In [ ]:
import numpy as np
from sklearn.covariance import GraphicalLassoCV

from regain import utils

In [ ]:
utils.save_pickle(lamda, "lambda_ctm.pkl")
lamda = utils.load_pickle('lambda_ctm.pkl')

In [ ]:
dff.values.shape

In [ ]:
gl = GraphicalLassoCV().fit(lamda)

In [ ]:
p = gl.precision_

A = np.abs(p - np.diag(np.diag(p)))
A = retain_top_n(A, 20)
G = nx.from_numpy_matrix(A * 3)
fig = pl.plot_circular(G, topic_str_repr, 2, cmap='Blues')

fig['layout'].update(height=800, width=800,hovermode='closest',
                     paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)')
# fig.layout.annotations += tuple([dict(
#     text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
#     showarrow=False, xref="paper", yref="paper", x=0.005, y=-0.2)])
# fig['layout'].update(scene=dict(aspectmode="data"))
py.iplot(fig)
# py.offline.iplot(fig, filename='figure_factory_subplot', image='svg')